### Download Dataset

In [ ]:
import opendatasets as od
import pandas as pd

In [5]:
od.download("https://www.kaggle.com/datasets/marshalpatel3558/diabetes-prediction-dataset-legit-dataset","../data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/marshalpatel3558/diabetes-prediction-dataset-legit-dataset


100%|██████████| 15.6k/15.6k [00:00<00:00, 4.42MB/s]

In [7]:
diabetes_dataset = pd.read_csv("../data/diabetes-prediction-dataset-legit-dataset/Dataset of Diabetes .csv")

In [ ]:
diabetes_dataset.head(5)

In [9]:
diabetes_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         1000 non-null   int64  
 1   No_Pation  1000 non-null   int64  
 2   Gender     1000 non-null   object 
 3   AGE        1000 non-null   int64  
 4   Urea       1000 non-null   float64
 5   Cr         1000 non-null   int64  
 6   HbA1c      1000 non-null   float64
 7   Chol       1000 non-null   float64
 8   TG         1000 non-null   float64
 9   HDL        1000 non-null   float64
 10  LDL        1000 non-null   float64
 11  VLDL       1000 non-null   float64
 12  BMI        1000 non-null   float64
 13  CLASS      1000 non-null   object 
dtypes: float64(8), int64(4), object(2)
memory usage: 109.5+ KB


In [15]:
list_class = diabetes_dataset['CLASS'].unique()
list_gender = diabetes_dataset['Gender'].unique()
print(f'The dataset present some errors like diferents terms to same class.\nFor label CLASS: {list_class}\nFor label Gender: {list_gender}')

The dataset present some errors like diferents terms to same class.
For label CLASS: ['N' 'N ' 'P' 'Y' 'Y ']
For label Gender: ['F' 'M' 'f']


In [18]:
diabetes_dataset['CLASS'] = diabetes_dataset['CLASS'].apply(lambda x: str.upper(x).strip())
diabetes_dataset['Gender'] = diabetes_dataset['Gender'].apply(lambda x: str.upper(x))

In [19]:
list_class = diabetes_dataset['CLASS'].unique()
list_gender = diabetes_dataset['Gender'].unique()
print(f'The dataset present some errors like diferents terms to same class.\nFor label CLASS: {list_class}\nFor label Gender: {list_gender}')

The dataset present some errors like diferents terms to same class.
For label CLASS: ['N' 'P' 'Y']
For label Gender: ['F' 'M']


### Apply XGBoost on the Dataset

### References

[Dive into XGBoost and Scikit-learn: Machine Learning with XGBoost and Scikit-learn](https://medium.com/@bravinwasike18/dive-into-xgboost-and-scikit-learnmachine-learning-with-xgboost-and-scikit-learn-17e2cf54f3a3)